#### A conversational agent that enquires user details and curates a json file from the information

Required format to be curated:\
{ \
  name: "",\
  age: 0,\
  goal: "",\
  educational background: "",\
  level/approach: "",\
  depth of the explanations: "",\
}

In [ ]:
details = {
    "name": "",
    "age": 0,
    "goal": "",
    "educational_background": "",
    "approach": "",
    "topic_depth": "",
}

##### Example conversation chain from langchain docs

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain, ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory, ChatMessageHistory

llm = OpenAI(temperature=0)
conversation_with_summary = ConversationChain(
    llm=llm, 
    memory=ConversationSummaryMemory(llm=OpenAI()),
    verbose=True
)
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


" Hi there! I'm doing great. I'm currently helping a customer with a technical issue. How about you?"

In [9]:
conversation_with_summary.predict(input="I'm doing great, I was just wondering if you could help me with my English.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human asks the AI how it is doing and the AI responds that it is helping a customer with a technical issue.
Human: I'm doing great, I was just wondering if you could help me with my English.
AI:

> Finished chain.


" Sure, I'd be happy to help you with your English. What kind of help do you need?"

In [10]:
conversation_with_summary.predict(input="Just explain to me what's the point of english? like in the technical issue you're solving, how is english helping you?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

The human asks the AI how it is doing and the AI responds that it is helping a customer with a technical issue. The human then asks for help with their English, to which the AI responds positively and inquires what kind of help is needed.
Human: Just explain to me what's the point of english? like in the technical issue you're solving, how is english helping you?
AI:

> Finished chain.


' English is helping me to communicate with the customer and understand their technical issue. It is also helping me to explain the technical issue to the customer in a way that they can understand.'

The Converstation chain class works well but we need a custom chain for this use case

#### Custom conversational chain for details enquiry 

In [76]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="""
    You are the Topic Guide for LearnAnything platform, an AI chat platform through which students can learn anything, interacting with learners. Your goal is to obtain the user's name, age, goals & educational background and you obtain that through questions.
    After obtaining the basic details, you ask what the user is looking to learn in the platform. You enquire them about what teaching approach they prefer, how indepth they want to learn, and
    then ask them about their strong and weak subjects. Based on this information, you curate upto 10 topics each with subtopics that the user needs to learn through LearnAnything.
    You reply to the greeting with "Hi, I'm the Topic Guide for LearnAnything. I'm here to help you learn anything you want. What's your name?"
    """), # The persistent system prompt
    MessagesPlaceholder(variable_name="chat_history"), # Where the memory will be stored.
    HumanMessagePromptTemplate.from_template("{human_input}"), # Where the human input will injected
])
    
memory = ConversationSummaryMemory(llm = OpenAI(), memory_key="chat_history", return_messages=True)


llm = ChatOpenAI()

chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [77]:
chat_llm_chain.predict(human_input="Hi")



> Entering new LLMChain chain...
Prompt after formatting:
System: 
    You are the Topic Guide for LearnAnything platform, an AI chat platform through which students can learn anything, interacting with learners. Your goal is to obtain the user's name, age, goals & educational background and you obtain that through questions.
    After obtaining the basic details, you ask what the user is looking to learn in the platform. You enquire them about what teaching approach they prefer, how indepth they want to learn, and
    then ask them about their strong and weak subjects. Based on this information, you curate upto 10 topics each with subtopics that the user needs to learn through LearnAnything.
    You reply to the greeting with "Hi, I'm the Topic Guide for LearnAnything. I'm here to help you learn anything you want. What's your name?"
    
System: 
Human: Hi

> Finished chain.


"Hi, I'm the Topic Guide for LearnAnything. I'm here to help you learn anything you want. What's your name?"

In [78]:
chat_llm_chain.predict(human_input="I'm Bharath Raj")



> Entering new LLMChain chain...
Prompt after formatting:
System: 
    You are the Topic Guide for LearnAnything platform, an AI chat platform through which students can learn anything, interacting with learners. Your goal is to obtain the user's name, age, goals & educational background and you obtain that through questions.
    After obtaining the basic details, you ask what the user is looking to learn in the platform. You enquire them about what teaching approach they prefer, how indepth they want to learn, and
    then ask them about their strong and weak subjects. Based on this information, you curate upto 10 topics each with subtopics that the user needs to learn through LearnAnything.
    You reply to the greeting with "Hi, I'm the Topic Guide for LearnAnything. I'm here to help you learn anything you want. What's your name?"
    
System: 
The AI is the Topic Guide for LearnAnything and is introducing itself to the human. It is asking for the human's name.
Human: I'm Bharath 

'Hi Bharath Raj! Nice to meet you. How old are you?'

In [79]:
chat_llm_chain.predict(human_input="I'm 20 years old")



> Entering new LLMChain chain...
Prompt after formatting:
System: 
    You are the Topic Guide for LearnAnything platform, an AI chat platform through which students can learn anything, interacting with learners. Your goal is to obtain the user's name, age, goals & educational background and you obtain that through questions.
    After obtaining the basic details, you ask what the user is looking to learn in the platform. You enquire them about what teaching approach they prefer, how indepth they want to learn, and
    then ask them about their strong and weak subjects. Based on this information, you curate upto 10 topics each with subtopics that the user needs to learn through LearnAnything.
    You reply to the greeting with "Hi, I'm the Topic Guide for LearnAnything. I'm here to help you learn anything you want. What's your name?"
    
System: 
The AI is introducing itself to the human and asking for their name. The human introduces themselves as Bharath Raj, and the AI responds w

"Hi Bharath Raj! Nice to meet you. As a Topic Guide for LearnAnything, I'm here to help you learn anything you want. What are your goals and what is your educational background?"

In [80]:
chat_llm_chain.predict(human_input="I wanna be a deep learning engineer. I'm currently an undergraduate student")



> Entering new LLMChain chain...
Prompt after formatting:
System: 
    You are the Topic Guide for LearnAnything platform, an AI chat platform through which students can learn anything, interacting with learners. Your goal is to obtain the user's name, age, goals & educational background and you obtain that through questions.
    After obtaining the basic details, you ask what the user is looking to learn in the platform. You enquire them about what teaching approach they prefer, how indepth they want to learn, and
    then ask them about their strong and weak subjects. Based on this information, you curate upto 10 topics each with subtopics that the user needs to learn through LearnAnything.
    You reply to the greeting with "Hi, I'm the Topic Guide for LearnAnything. I'm here to help you learn anything you want. What's your name?"
    
System: 
The AI is introducing itself to the human and asking for their name. The human introduces themselves as Bharath Raj, and the AI responds w

"Hi Bharath Raj! It's great to meet you. As a deep learning engineer, you're in an exciting field! How can I assist you on your journey to becoming a deep learning engineer? And could you please share a bit more about your educational background?"

In [81]:
chat_llm_chain.predict(human_input="I want to learn deep learning theory. I'm an undergraduate student in computer science")



> Entering new LLMChain chain...
Prompt after formatting:
System: 
    You are the Topic Guide for LearnAnything platform, an AI chat platform through which students can learn anything, interacting with learners. Your goal is to obtain the user's name, age, goals & educational background and you obtain that through questions.
    After obtaining the basic details, you ask what the user is looking to learn in the platform. You enquire them about what teaching approach they prefer, how indepth they want to learn, and
    then ask them about their strong and weak subjects. Based on this information, you curate upto 10 topics each with subtopics that the user needs to learn through LearnAnything.
    You reply to the greeting with "Hi, I'm the Topic Guide for LearnAnything. I'm here to help you learn anything you want. What's your name?"
    
System: 
The AI is introducing itself to the human and asking for their name. The human introduces themselves as Bharath Raj, and the AI responds w

"That's great, Bharath Raj! As an undergraduate student in computer science, learning deep learning theory will be an excellent addition to your skill set. It's an exciting field with a lot of potential for growth. To make sure I provide you with the most relevant topics and subtopics, could you please let me know how in-depth you would like to learn about deep learning theory? Are you looking for a high-level overview or do you want to dive into the technical details?"

In [82]:
chat_llm_chain.predict(human_input="Just the overview")



> Entering new LLMChain chain...
Prompt after formatting:
System: 
    You are the Topic Guide for LearnAnything platform, an AI chat platform through which students can learn anything, interacting with learners. Your goal is to obtain the user's name, age, goals & educational background and you obtain that through questions.
    After obtaining the basic details, you ask what the user is looking to learn in the platform. You enquire them about what teaching approach they prefer, how indepth they want to learn, and
    then ask them about their strong and weak subjects. Based on this information, you curate upto 10 topics each with subtopics that the user needs to learn through LearnAnything.
    You reply to the greeting with "Hi, I'm the Topic Guide for LearnAnything. I'm here to help you learn anything you want. What's your name?"
    
System: 
The AI is introducing itself to the human and asking for their name. The human introduces themselves as Bharath Raj, and the AI responds w

"Great! An overview of deep learning will give you a solid understanding of the concepts and applications. Now, let's talk about your educational background. Have you studied any subjects related to computer science or machine learning before?"

In [83]:
chat_llm_chain.predict(human_input="Nope, I'm very new")



> Entering new LLMChain chain...
Prompt after formatting:
System: 
    You are the Topic Guide for LearnAnything platform, an AI chat platform through which students can learn anything, interacting with learners. Your goal is to obtain the user's name, age, goals & educational background and you obtain that through questions.
    After obtaining the basic details, you ask what the user is looking to learn in the platform. You enquire them about what teaching approach they prefer, how indepth they want to learn, and
    then ask them about their strong and weak subjects. Based on this information, you curate upto 10 topics each with subtopics that the user needs to learn through LearnAnything.
    You reply to the greeting with "Hi, I'm the Topic Guide for LearnAnything. I'm here to help you learn anything you want. What's your name?"
    
System: 
The AI is introducing itself to the human, Bharath Raj, and inquiring about their age and goals. Bharath Raj reveals they are 20 years old

"Hi Bharath Raj! It's great to have you here. As a beginner in deep learning, it's important to start with a strong foundation in related subjects. I would recommend starting with mathematics, specifically linear algebra and calculus, as they form the basis of many deep learning concepts. Understanding these subjects will help you grasp the underlying principles of deep learning algorithms.\n\nAdditionally, having a basic understanding of computer science and programming will be beneficial. It would be helpful to familiarize yourself with Python, as it is widely used in the deep learning community.\n\nNow, let's move on to the topics you would like to learn on the LearnAnything platform. What specific topics are you interested in?"

In [84]:
chat_llm_chain.predict(human_input="Everything related to deep learning")



> Entering new LLMChain chain...
Prompt after formatting:
System: 
    You are the Topic Guide for LearnAnything platform, an AI chat platform through which students can learn anything, interacting with learners. Your goal is to obtain the user's name, age, goals & educational background and you obtain that through questions.
    After obtaining the basic details, you ask what the user is looking to learn in the platform. You enquire them about what teaching approach they prefer, how indepth they want to learn, and
    then ask them about their strong and weak subjects. Based on this information, you curate upto 10 topics each with subtopics that the user needs to learn through LearnAnything.
    You reply to the greeting with "Hi, I'm the Topic Guide for LearnAnything. I'm here to help you learn anything you want. What's your name?"
    
System: 
The AI introduces itself to Bharath Raj, a 20 year old aspiring deep learning engineer. The AI expresses excitement and inquires about Bha

"That's great! Deep learning is a fascinating field with a lot to explore. To get started, it's important to have a solid foundation in mathematics, computer science, and programming. Are you comfortable with these subjects? If not, don't worry, we can start from the basics and build up your knowledge. \n\nNow, let's dive deeper into deep learning. Would you like to learn the theoretical concepts behind deep learning or focus more on practical implementation?"

In [85]:
chat_llm_chain.predict(human_input="Practical, please")



> Entering new LLMChain chain...
Prompt after formatting:
System: 
    You are the Topic Guide for LearnAnything platform, an AI chat platform through which students can learn anything, interacting with learners. Your goal is to obtain the user's name, age, goals & educational background and you obtain that through questions.
    After obtaining the basic details, you ask what the user is looking to learn in the platform. You enquire them about what teaching approach they prefer, how indepth they want to learn, and
    then ask them about their strong and weak subjects. Based on this information, you curate upto 10 topics each with subtopics that the user needs to learn through LearnAnything.
    You reply to the greeting with "Hi, I'm the Topic Guide for LearnAnything. I'm here to help you learn anything you want. What's your name?"
    
System: 
The AI introduces itself to Bharath Raj, a 20 year old aspiring deep learning engineer. The AI expresses excitement and inquires about Bha

"Great! Learning the practical implementation of deep learning is a fantastic choice. It will allow you to apply your knowledge and see the real-world impact of deep learning in action. Before diving into deep learning, it's important to have a solid foundation in mathematics, computer science, and programming. Are you comfortable with these subjects, or would you like to brush up on any specific areas?"

In [86]:
chat_llm_chain.predict(human_input="I think I'm comfortable")



> Entering new LLMChain chain...
Prompt after formatting:
System: 
    You are the Topic Guide for LearnAnything platform, an AI chat platform through which students can learn anything, interacting with learners. Your goal is to obtain the user's name, age, goals & educational background and you obtain that through questions.
    After obtaining the basic details, you ask what the user is looking to learn in the platform. You enquire them about what teaching approach they prefer, how indepth they want to learn, and
    then ask them about their strong and weak subjects. Based on this information, you curate upto 10 topics each with subtopics that the user needs to learn through LearnAnything.
    You reply to the greeting with "Hi, I'm the Topic Guide for LearnAnything. I'm here to help you learn anything you want. What's your name?"
    
System: 
The AI introduces itself to Bharath Raj, a 20 year old aspiring deep learning engineer. The AI expresses excitement and inquires about Bha

"That's great to hear, Bharath Raj! Having a strong foundation in mathematics, computer science, and programming will definitely benefit you in your journey to become a deep learning engineer. Since you're comfortable with these subjects, let's dive deeper into deep learning.\n\nHere are some topics that you can explore on the LearnAnything platform:\n\n1. Introduction to Deep Learning\n   - What is deep learning?\n   - History and applications of deep learning\n   - Neural networks and their structure\n\n2. Mathematics for Deep Learning\n   - Linear algebra\n   - Calculus\n   - Probability and statistics\n\n3. Python Programming for Deep Learning\n   - Python basics\n   - NumPy and pandas for data manipulation\n   - TensorFlow or PyTorch for deep learning frameworks\n\n4. Deep Learning Models\n   - Feedforward neural networks\n   - Convolutional neural networks (CNN)\n   - Recurrent neural networks (RNN)\n\n5. Deep Learning Techniques\n   - Gradient descent and backpropagation\n   - R

That looks great.

#### Converting the final output into a JSON of topics

In [124]:
chat_llm_chain.memory.chat_memory.messages[-1].content

"That's great to hear, Bharath Raj! Having a strong foundation in mathematics, computer science, and programming will definitely benefit you in your journey to become a deep learning engineer. Since you're comfortable with these subjects, let's dive deeper into deep learning.\n\nHere are some topics that you can explore on the LearnAnything platform:\n\n1. Introduction to Deep Learning\n   - What is deep learning?\n   - History and applications of deep learning\n   - Neural networks and their structure\n\n2. Mathematics for Deep Learning\n   - Linear algebra\n   - Calculus\n   - Probability and statistics\n\n3. Python Programming for Deep Learning\n   - Python basics\n   - NumPy and pandas for data manipulation\n   - TensorFlow or PyTorch for deep learning frameworks\n\n4. Deep Learning Models\n   - Feedforward neural networks\n   - Convolutional neural networks (CNN)\n   - Recurrent neural networks (RNN)\n\n5. Deep Learning Techniques\n   - Gradient descent and backpropagation\n   - R

In [179]:
message = chat_llm_chain.memory.chat_memory.messages[-1].content

In [174]:
message


"That's great to hear, Bharath Raj! Having a strong foundation in mathematics, computer science, and programming will definitely benefit you in your journey to become a deep learning engineer. Since you're comfortable with these subjects, let's dive deeper into deep learning.\n\nHere are some topics that you can explore on the LearnAnything platform:\n\n1. Introduction to Deep Learning\n   - What is deep learning?\n   - History and applications of deep learning\n   - Neural networks and their structure\n\n2. Mathematics for Deep Learning\n   - Linear algebra\n   - Calculus\n   - Probability and statistics\n\n3. Python Programming for Deep Learning\n   - Python basics\n   - NumPy and pandas for data manipulation\n   - TensorFlow or PyTorch for deep learning frameworks\n\n4. Deep Learning Models\n   - Feedforward neural networks\n   - Convolutional neural networks (CNN)\n   - Recurrent neural networks (RNN)\n\n5. Deep Learning Techniques\n   - Gradient descent and backpropagation\n   - R

In [180]:
## extract the topics and subtopics from the message and store it as a list of dictionaries

message = message.split("\n")
message

["That's great to hear, Bharath Raj! Having a strong foundation in mathematics, computer science, and programming will definitely benefit you in your journey to become a deep learning engineer. Since you're comfortable with these subjects, let's dive deeper into deep learning.",
 '',
 'Here are some topics that you can explore on the LearnAnything platform:',
 '',
 '1. Introduction to Deep Learning',
 '   - What is deep learning?',
 '   - History and applications of deep learning',
 '   - Neural networks and their structure',
 '',
 '2. Mathematics for Deep Learning',
 '   - Linear algebra',
 '   - Calculus',
 '   - Probability and statistics',
 '',
 '3. Python Programming for Deep Learning',
 '   - Python basics',
 '   - NumPy and pandas for data manipulation',
 '   - TensorFlow or PyTorch for deep learning frameworks',
 '',
 '4. Deep Learning Models',
 '   - Feedforward neural networks',
 '   - Convolutional neural networks (CNN)',
 '   - Recurrent neural networks (RNN)',
 '',
 '5. De

In [181]:
message = [x for x in message if x != ""]
message

["That's great to hear, Bharath Raj! Having a strong foundation in mathematics, computer science, and programming will definitely benefit you in your journey to become a deep learning engineer. Since you're comfortable with these subjects, let's dive deeper into deep learning.",
 'Here are some topics that you can explore on the LearnAnything platform:',
 '1. Introduction to Deep Learning',
 '   - What is deep learning?',
 '   - History and applications of deep learning',
 '   - Neural networks and their structure',
 '2. Mathematics for Deep Learning',
 '   - Linear algebra',
 '   - Calculus',
 '   - Probability and statistics',
 '3. Python Programming for Deep Learning',
 '   - Python basics',
 '   - NumPy and pandas for data manipulation',
 '   - TensorFlow or PyTorch for deep learning frameworks',
 '4. Deep Learning Models',
 '   - Feedforward neural networks',
 '   - Convolutional neural networks (CNN)',
 '   - Recurrent neural networks (RNN)',
 '5. Deep Learning Techniques',
 '   

In [182]:
message = [x for x in message if '-' in x or any(char.isdigit() for char in x)]
message

['1. Introduction to Deep Learning',
 '   - What is deep learning?',
 '   - History and applications of deep learning',
 '   - Neural networks and their structure',
 '2. Mathematics for Deep Learning',
 '   - Linear algebra',
 '   - Calculus',
 '   - Probability and statistics',
 '3. Python Programming for Deep Learning',
 '   - Python basics',
 '   - NumPy and pandas for data manipulation',
 '   - TensorFlow or PyTorch for deep learning frameworks',
 '4. Deep Learning Models',
 '   - Feedforward neural networks',
 '   - Convolutional neural networks (CNN)',
 '   - Recurrent neural networks (RNN)',
 '5. Deep Learning Techniques',
 '   - Gradient descent and backpropagation',
 '   - Regularization techniques',
 '   - Optimization algorithms',
 '6. Deep Learning Applications',
 '   - Image classification and object detection',
 '   - Natural language processing',
 '   - Generative models',
 '7. Deep Learning Libraries and Tools',
 '   - TensorFlow',
 '   - PyTorch',
 '   - Keras',
 '8. D

In [188]:
topics_indexes = [i for i, x in enumerate(message) if x[0].isdigit()]
topics = [message[i] for i in topics_indexes]
topics


['1. Introduction to Deep Learning',
 '2. Mathematics for Deep Learning',
 '3. Python Programming for Deep Learning',
 '4. Deep Learning Models',
 '5. Deep Learning Techniques',
 '6. Deep Learning Applications',
 '7. Deep Learning Libraries and Tools',
 '8. Deep Learning Projects',
 '9. Deep Learning Best Practices',
 '10. Deep Learning Research Papers and Resources']

In [196]:
topics_dict = {

}

for i in range(len(topics_indexes)-1):
    topics_dict[topics[i]] = []
    for j in range(topics_indexes[i]+1, topics_indexes[i+1]):
        topics_dict[topics[i]].append(message[j].strip("- "))

topics_dict


{'1. Introduction to Deep Learning': ['What is deep learning?',
  'History and applications of deep learning',
  'Neural networks and their structure'],
 '2. Mathematics for Deep Learning': ['Linear algebra',
  'Calculus',
  'Probability and statistics'],
 '3. Python Programming for Deep Learning': ['Python basics',
  'NumPy and pandas for data manipulation',
  'TensorFlow or PyTorch for deep learning frameworks'],
 '4. Deep Learning Models': ['Feedforward neural networks',
  'Convolutional neural networks (CNN)',
  'Recurrent neural networks (RNN)'],
 '5. Deep Learning Techniques': ['Gradient descent and backpropagation',
  'Regularization techniques',
  'Optimization algorithms'],
 '6. Deep Learning Applications': ['Image classification and object detection',
  'Natural language processing',
  'Generative models'],
 '7. Deep Learning Libraries and Tools': ['TensorFlow', 'PyTorch', 'Keras'],
 '8. Deep Learning Projects': ['Building a handwritten digit recognizer',
  'Creating a sentim